In [46]:
import sagemaker
import numpy as np
import pandas as pd
import os
import torch
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [47]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Deploy model

In [54]:
class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data='s3://sagemaker-eu-west-1-087816224558/sagemaker-pytorch-2020-05-31-18-26-11-789/output/model.tar.gz',
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)

In [55]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

---------------------

KeyboardInterrupt: 

In [41]:
predictions = predictor.predict('I live in USA, this is my IBM laptop.')

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-pytorch-2020-05-31-19-10-35-513 in account 087816224558 for more information.

In [ ]:
print(predictions)